In [1]:
import os
import torch
device='cuda' if torch.cuda.is_available() else 'cpu' 
from tqdm.notebook import tqdm
from transformers import BertForSequenceClassification, BertTokenizer,BertConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,load_metric

import torch.quantization
import torch.nn as nn

In [15]:
train_dataset = load_dataset('glue', 'sst2', split='train') # challenge, sst2
val_dataset = load_dataset('glue', 'sst2', split='validation')
test_dataset = load_dataset('glue', 'sst2', split='test')

Reusing dataset glue (/home/as14229/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/home/as14229/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/home/as14229/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [16]:
train_dataset = train_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
val_dataset = val_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
test_dataset = test_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

val_dataset = val_dataset.remove_columns(['label'])
test_dataset = test_dataset.remove_columns(['label'])
train_dataset = train_dataset.remove_columns(['label'])

Loading cached processed dataset at /home/as14229/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-882614e3a3716745.arrow
Loading cached processed dataset at /home/as14229/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7b1f5eac58ea78a8.arrow
Loading cached processed dataset at /home/as14229/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e62954b2d2f2ea2e.arrow


In [4]:
# tokenizer = BertTokenizer.from_pretrained('saved_model')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
MAX_LENGTH = 128
train_dataset = train_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
val_dataset = val_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
test_dataset = test_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)

In [7]:
train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [8]:
train_dataset

Dataset({
    features: ['sentence', 'idx', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 67349
})

In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [9]:
model = BertForSequenceClassification.from_pretrained('./bert_main_sst_model', return_dict=True)
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', return_dict=True)

training_args = TrainingArguments(
    output_dir='./results',      
    overwrite_output_dir = 'True',#output directory
    learning_rate=1e-5,
    num_train_epochs=2,
    warmup_steps = 600,
    weight_decay=0.01,
    per_device_train_batch_size=32,                #batch size per device during training
    per_device_eval_batch_size=32,                #batch size for evaluation
    logging_dir='./logs',            
    logging_steps=100,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    load_best_model_at_end=False,
    save_total_limit = 1,
    save_strategy = "no",
    seed=42,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.180800,0.253846,0.918578,0.918554,0.918539,0.918572
2,0.120100,0.252820,0.923165,0.923129,0.923203,0.923076


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 32
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




In [13]:
train_out = trainer.train()
# trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 32


{'eval_loss': 0.2528201937675476,
 'eval_accuracy': 0.9231651376146789,
 'eval_f1': 0.9231286421595136,
 'eval_precision': 0.9232030530332939,
 'eval_recall': 0.9230761134966743,
 'eval_runtime': 2.4346,
 'eval_samples_per_second': 358.171,
 'eval_steps_per_second': 11.501}

In [14]:
# model.save_pretrained('./saved_model/')
# tokenizer.save_pretrained('./saved_model/')

In [5]:
############# QUANTIZATION

In [6]:
import torch
import torch.nn as nn
import os
from transformers import BertForSequenceClassification, BertTokenizer,BertConfig

/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (MB):', size/1e6)
    os.remove('temp.p')
    return size

In [8]:
model = BertForSequenceClassification.from_pretrained('bert_main_sst_model', return_dict =True)
quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Linear}, dtype=torch.qint8
)
model_size = print_size_of_model(model,"fp32")
quantized_model_size = print_size_of_model(quantized_model,"int8")

print("{0:.2f} times smaller".format(model_size/quantized_model_size))

model:  fp32  	 Size (MB): 438.016429
model:  int8  	 Size (MB): 181.496197
2.41 times smaller


In [10]:
from datasets import load_dataset,load_metric
MAX_LENGTH = 128
tokenizer = BertTokenizer.from_pretrained('./bert_main_sst_model')

val_dataset = load_dataset('glue', 'sst2', split='validation')
val_dataset = val_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
val_dataset = val_dataset.remove_columns(['label'])
val_dataset = val_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)

Reusing dataset glue (/home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-80a8390bc04583f0.arrow
Loading cached processed dataset at /home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-1271d2b78f498c8f.arrow


In [32]:
input_ids =  torch.tensor(val_dataset['input_ids'][:0])
token_type_ids = torch.tensor(val_dataset['token_type_ids'][:0])
attention_mask = torch.tensor(val_dataset['attention_mask'][:0])
dummy_input = [input_ids, attention_mask, token_type_ids]
traced_model = torch.jit.trace(quantized_model, dummy_input, strict=False)

torch.jit.save(traced_model, "bert_quantized8_sst_model/model.pt")

/state/partition1/job-19337151/ipykernel_1806277/3651675027.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids =  torch.tensor(val_dataset['input_ids'][:0])
/state/partition1/job-19337151/ipykernel_1806277/3651675027.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token_type_ids = torch.tensor(val_dataset['token_type_ids'][:0])
/state/partition1/job-19337151/ipykernel_1806277/3651675027.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(val_dataset['attention_mask'][:0])
